In [1]:
import furuta_sympy_calc as furu_calc
import pandas as pd
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp

# ==========================================================
# Parameter (flach, ohne Klassen)
# ==========================================================
g = 9.81

# Geometrie
L1 = 0.155
L2 = 0.240
lP = 0.1917

# Massen
mS = 0.0274
mP = 0.0  # 0.0510   # <- 0.0 wenn keine Zusatzmasse montiert ist
m2 = mS + mP

# Trägheitsmomente
J1_hat = 2.89e-2
J0_hat = J1_hat + m2 * L1**2

l2 = L2 / 2.0
J2_hat = (1 / 12) * mS * L2**2 + mS * l2**2 + mP * lP**2

# Reibungsparameter
mu_V1 = 50.91e-2
mu_V2 = 1.332e-5
haft_reibung = False
if haft_reibung:
    mu_H1 = 93.58e-3
    mu_H2 = 3.018e-4
else:
    mu_H1 = 0.0
    mu_H2 = 0.0

epsilon = 0.01
tau_max = 4.0

#sympy symbols
th1 = furu_calc.th1
th1d = furu_calc.th1d
th2 = furu_calc.th2
th2d = furu_calc.th2d
tau = furu_calc.tau

In [2]:
# non linear expressions
th1dd_expr, th2dd_expr = furu_calc.make_non_linear_furuta_accel_expr(
    g=g, L1=L1, l2=l2, J0_hat=J0_hat, J2_hat=J2_hat, m2=m2,
    mu_V1=mu_V1, mu_H1=mu_H1, mu_V2=mu_V2, mu_H2=mu_H2,
    epsilon=epsilon,
    simplify_expr=False,   # wenn's beim Start zu lange dauert -> False
)

In [3]:
f1 = th1d
f2 = th1dd_expr
f3 = th2d
f4 = th2dd_expr

f = sp.Matrix([f1, f2, f3, f4])
x = sp.Matrix([th1, th1d, th2, th2d])
u = sp.Matrix([tau])

A = f.jacobian(x)
b = f.jacobian(u)

A_lin = A.subs({th1:0, th1d:0, th2:0, th2d:0, tau:0})
b_lin = b.subs({th1:0, th1d:0, th2:0, th2d:0, tau:0})

A_lin

Matrix([
[0,                 1,                 0,                    0],
[0, -17.5161709980497,  1.07509937893889, 0.000443968358900185],
[0,                 0,                 0,                    1],
[0,  16.9687906543607, -62.3540025233471,   -0.025749437413378]])

In [4]:
b_lin

Matrix([
[                0],
[  34.406150064918],
[                0],
[-33.3309578753893]])